In [ ]:
%matplotlib inline

import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({'font.size':16})

import sciunit
from owtests import OW_HOME
import wcon
import open_worm_analysis_toolbox as owat

In [ ]:
OWAT_TESTS = os.path.join(OW_HOME,'tests','owtests','open-worm-analysis-toolbox')

# Model classes

In [ ]:
class HasFeatures(sciunit.Capability):
    def get_feature_stat(self, feature, stat):
        raise NotImplementedError()

In [ ]:
class MovementModel(sciunit.Model,HasFeatures):
    """Model class for testing worm movement"""
    
    def __init__(self, path, name=None, attrs=None):
        super(MovementModel,self).__init__(name)
        self.path = path
        # Only needed for API compliance
        self.attrs = attrs if attrs else {}
        self.run_params = {}
        self._url = 'http://github.com/openworm/tests'
           
    # Mapping between traditional feature names and OWAT feature names
    # for ten core features
    feature_map = \
        {'area':None,
         'forward_frequency':'locomotion.motion_events.forward.frequency',
         'backward_frequency':'locomotion.motion_events.backward.frequency',
         'head_tip_speed_abs':'locomotion.velocity.head_tip.speed',
         'length':'morphology.length',
         'max_amplitude':'posture.amplitude_max',
         'midbody_bend_mean_abs':'posture.bends.midbody.mean',
         'midbody_speed_abs':'locomotion.velocity.midbody.speed',
         'path_range':None,
         'paused_frequency':'locomotion.motion_events.paused.frequency'}
    
    def extract_features(self):
        self.features = owat.WormFeatures.from_disk(self.path)
        
    def compute_feature_stats(self):
        feature_list = list(self.features._features.keys())
        # Make a data frame to store feature stats
        self.feature_stats = pd.DataFrame(index=feature_list,columns=['mean'])
        for stat in list(self.feature_stats):
            n_valid = 0
            for feature,value in self.features._features.items():
                try: # Fill with feature means only if 
                     # the feature is found and has some numeric data
                    f = getattr(pd.Series(value.value),stat)
                    result = f()
                    if not np.isnan(result):
                        self.feature_stats.loc[feature,stat] = result
                        n_valid += 1
                except:
                    pass
            # Valid features are the ones that were computed and have some numeric data
            print("%d valid feature values found for %s" % (n_valid,stat))
        self.feature_stats = self.feature_stats.astype('float')
        
    def get_feature_stat(self, feature, stat):
        s = self.feature_stats[stat]
        if feature in s:
            result = s[feature]
        elif feature in self.feature_map:
            alt_feature = self.feature_map[feature]
            if alt_feature in s:
                result = s[alt_feature]
            else:
                raise KeyError("Couldn't find an entry for %s or %s" % (feature,alt_feature))
        else:
            raise KeyError("Couldn't find an entry for %s" % feature)
        return result
        
    def get_amplitude_ratio(self):
        return self.get_feature_stat('posture.amplitude_ratio',
                                     'mean')

In [ ]:
import pandas as pd
path = os.path.join(OWAT_TESTS,'experiment','movement_feature_means_all.csv')
df = pd.read_csv(path,index_col=0)
for col in MovementModel.feature_map.keys():
    try:
        df = df[df[col] != 'None']
        df[col] = df[col].astype('float')
    except:
        pass
df.head()

In [ ]:
df.hist(['amplitude_ratio']);

In [ ]:
#model_path = os.path.join(OWAT_TESTS,'model','worm_motion_log.wcon')
#model_long = MovementModel(model_path,name='Long Model')
#%time model_long.extract_features()
#%time model_long.compute_feature_stats()

model_path = os.path.join(OWAT_TESTS,'model','worm_motion_log_short.wcon')
model_short = MovementModel(model_path,name='sibernetic_c302_short')
%time model_short.extract_features()
%time model_short.compute_feature_stats()

#models = [model_long, model_short]
models = [model_short]

In [ ]:
model_short.feature_stats.loc['morphology.length']

In [ ]:
class MovementFeatureTest(sciunit.Test): 
    """Base class for movement feature tests"""
    
    score_type = sciunit.scores.ZScore   
    attribute = None
    
    def validate_observation(self, observation):
        for col in ['mean','std']:
            s = observation['df'][self.attribute]
            
            f = getattr(observation['df'][self.attribute],col)
            print(self.attribute,col)
            observation[col] = f()
        return observation
        
    def generate_prediction(self, model):
        mean = model.get_feature_stat(self.attribute,'mean')
        if self.name.endswith('absTest'):
            mean = np.abs(mean)
        return mean#{'mean': mean}
    
def test_factory(attribute):
    class C(MovementFeatureTest):
        attribute = attribute
    C.__name__ = '%sTest' % attribute
    return C

In [ ]:
tests = []
for attribute in MovementModel.feature_map:
    if MovementModel.feature_map[attribute] is not None:
        cls = test_factory(attribute)
        tests.append(cls(observation={'df':df}))
suite = sciunit.TestSuite(tests)

In [ ]:
score_matrix = suite.judge(models,stop_on_error=False)

In [ ]:
for score in score_matrix[models[0]]:
    print(score.test.name,score.observation['mean'],score.prediction)

In [ ]:
score_matrix.T

In [ ]:
#from scidash_api import client
#client_instance = client.ScidashClient({'base_url': 'http://scidash.a88dc6aa.svc.dockerapp.io:8000'}, 
#                                       hostname="Workuity Dell")
#client_instance.login(username='openworm', password='passworm')

In [ ]:
#for test in suite.tests:
#    try:
#        del test.observation['df']
#    except:
#        pass

In [ ]:
#client_instance.upload_suite(suite,score_matrix)